## Explainable k-Medoids

In [31]:
import matplotlib.pyplot as plt
from sklearn import datasets
import mpl_toolkits.mplot3d 
from sklearn.cluster import KMeans
from sklearn import metrics
import numpy as np
import pandas as pd

In [32]:
iris = datasets.load_iris()
X = iris.data
y = iris.target

In [33]:
kmeans = KMeans(n_clusters=3, random_state=3).fit(X)

In [34]:
clusters = kmeans.cluster_centers_ 

In [35]:
preds = kmeans.predict(X)

In [36]:
def calc_feature_wise_distance_matrix(distance_metric, points, cluster_centers):
    
    centers_lst = []

    for i in points:
        centers_lst.append(np.array(cluster_centers))
    
    c = np.array(centers_lst)

    # calculate the distance of every feature value of ever obs to every feature value in every cluster.
    
    feature_wise_distance_matrix = []
    
    if distance_metric == "manhattan":
        for i,e in enumerate(X):
            feature_wise_distance_matrix.append(abs(c[i] - e))
        
    
    if distance_metric == "euclidean":
        for i,e in enumerate(X):
            feature_wise_distance_matrix.append((c[i] - e)**2)
    
    return np.array(feature_wise_distance_matrix)

In [37]:
A =calc_feature_wise_distance_matrix("manhattan", X, clusters)

In [38]:
def best_second_calc(feature_based_distances, preds):
    
    num_features = feature_based_distances.shape[2]
    
    assinged_cluster_list = []
    fb_distance_to_assinged_cluster_list = []
    
    best_alterantive_list = []
    fb_distance_to_best_alternative_list = []
    
    #for every obs:
    for idx, e in enumerate(feature_based_distances):
        #index of assinged cluster
        assigned_cluster = preds[idx]
        #feature-wise distances of point to assigned cluster
        distances_to_assigned = e[assigned_cluster]
        
        assinged_cluster_list.append(assigned_cluster)
        fb_distance_to_assinged_cluster_list.append(distances_to_assigned)
        
        #find best alternative:
        
        temp_bad = []
        temp_idx = []
        
        #for every feature
        for i in range(num_features):
            
            
            # best alternative: 
            best_alternative_distance = min(e[:,i])
            x = e[:,i].tolist()
            idx_best_alternative = x.index(best_alternative_distance)
            
            
            #if the best alternative is the assigned cluster, we have to find the second best alternative
            if idx_best_alternative == assigned_cluster:
                
                del x[idx_best_alternative]
                best_alternative_distance = min(x)
                idx_best_alternative = x.index(best_alternative_distance)
                
            temp_bad.append(best_alternative_distance)
            temp_idx.append(idx_best_alternative)

        best_alterantive_list.append(temp_idx)
        fb_distance_to_best_alternative_list.append(temp_bad)     
            
    return np.array(assinged_cluster_list), np.array(fb_distance_to_assinged_cluster_list), np.array(best_alterantive_list), np.array(fb_distance_to_best_alternative_list)

In [39]:
ac ,fb_ac ,ba, fb_ba = best_second_calc(A, preds)

## Relevance of feature j for point i:

$$R_{ji} := R_j(x_{ij},c_{kj},c_{k'_{ij}}) := \frac{d_j(x_{ij},c_{k'_{ij}}) - d_j(x_{ij},c_{kj})}{d_j(x_{ij},c_{k'_{ij}}) + d_j(x_{ij},c_{kj})}$$

$d_j(x_{ij},c_{k'_{ij}}$ is the distance to the best alternative.

$d_j(x_{ij},c_{kj})$ is the distance to the associated cluster.

In [40]:
# broadcasted: for every i, for every j
R = (fb_ba - fb_ac) / (fb_ba + fb_ac)

In [41]:
R

array([[ 0.79008788,  0.71102662,  0.95941809,  0.92811775],
       [ 0.80859739, -0.70625262,  0.95941809,  0.92811775],
       [ 0.59406025, -0.28698752,  0.9004776 ,  0.92811775],
       [ 0.52448239, -0.85145573,  0.97407513,  0.92811775],
       [ 0.9867785 ,  0.50738619,  0.95941809,  0.92811775],
       [ 0.1201556 ,  0.27290417,  0.83762847,  0.74071258],
       [ 0.52448239,  0.8419489 ,  0.95941809,  0.90908103],
       [ 0.9867785 ,  0.8419489 ,  0.97407513,  0.92811775],
       [ 0.42494184, -0.55382571,  0.95941809,  0.92811775],
       [ 0.80859739, -0.85145573,  0.97407513,  0.8026855 ],
       [ 0.1201556 ,  0.39442231,  0.97407513,  0.92811775],
       [ 0.68492204,  0.8419489 ,  0.9058518 ,  0.92811775],
       [ 0.68492204, -0.70625262,  0.95941809,  0.8026855 ],
       [ 0.38811228, -0.70625262,  0.8019449 ,  0.8026855 ],
       [-0.77308745,  0.23647604,  0.84835981,  0.92811775],
       [-0.54977669,  0.15416323,  0.97407513,  0.74071258],
       [ 0.1201556 ,  0.

## Global relevances

In [42]:
R_global = {"R_global_" + str(i) : np.sum(R[:,i]) / len(R) for i in range(R.shape[1])}  

In [43]:
R_global

{'R_global_0': 0.36975277918798033,
 'R_global_1': 0.13387253867696042,
 'R_global_2': 0.6688310755220815,
 'R_global_3': 0.5876523318789212}

## Cluster - wise relevances

In [44]:
df_c = pd.DataFrame(R)
df_c.rename({0:"R1", 1: "R2", 2: "R3", 3: "R4"}, axis=1, inplace = True)
df_c["assigned_clusters"] = preds

In [45]:
R_c = df_c.groupby(["assigned_clusters"]).mean()

In [46]:
R_c

,R1,R2,R3,R4
assigned_clusters,,,,
0,0.221561,0.161347,0.528592,0.444371
1,0.537161,0.153260,0.915832,0.869074
2,0.391266,0.063536,0.572641,0.451136


## Example Random Noise

In [47]:
df_X = pd.DataFrame(X)
df_X.rename({0:"X1", 1: "X2", 2: "X3", 3: "X4"}, axis=1, inplace = True)

In [48]:
X_5 = np.random.rand(150)
X_6 = np.random.rand(150)

In [49]:
df_X["X_5"] = X_5
df_X["X6"] = X_6

In [50]:
X = df_X.values
kmeans = KMeans(n_clusters=3, random_state=3).fit(X)

In [51]:
df_X

,X1,X2,X3,X4,X_5,X6
0,5.1,3.5,1.4,0.2,0.220475,0.135275
1,4.9,3.0,1.4,0.2,0.792414,0.367929
2,4.7,3.2,1.3,0.2,0.423428,0.531295
3,4.6,3.1,1.5,0.2,0.792134,0.549090
4,5.0,3.6,1.4,0.2,0.724471,0.446279
...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,0.502655,0.564005
146,6.3,2.5,5.0,1.9,0.535128,0.367379
147,6.5,3.0,5.2,2.0,0.438741,0.065502
148,6.2,3.4,5.4,2.3,0.473448,0.830066


In [52]:
kmeans.inertia_

103.64085666434812

In [53]:
### !! darf ich nicht so machen, da sonst fehler bei cluster zuordnung!

#prediction is "false" but only because the "label" is not correct, the class itself is correctly predicted. Maybe this needs to be solved later on differently.
preds = kmeans.predict(X)

In [55]:
centers = kmeans.cluster_centers_
print(centers)
print(centers.shape)

[[5.88360656 2.74098361 4.38852459 1.43442623 0.45132473 0.51246678]
 [5.006      3.428      1.462      0.246      0.42797707 0.54127521]
 [6.85384615 3.07692308 5.71538462 2.05384615 0.55184225 0.4529116 ]]
(3, 6)


In [56]:
A = calc_feature_wise_distance_matrix(distance_metric = "euclidean", points = X, cluster_centers=centers)

In [57]:
ac ,fb_ac ,ba, fb_ba = best_second_calc(A, preds)

In [58]:
# broadcasted: for every i, for every j
R = (fb_ba - fb_ac) / (fb_ba + fb_ac)

In [60]:
R_global = {"R_global_" + str(i) : np.sum(R[:,i]) / len(R) for i in range(R.shape[1])}  
R_global

{'R_global_0': 0.5146801555673295,
 'R_global_1': 0.20692686957001968,
 'R_global_2': 0.8028122539080923,
 'R_global_3': 0.7069228372678296,
 'R_global_4': -0.1852375414912179,
 'R_global_5': -0.08629074236335037}

Note, that the relevances $R_4$ and $R_5$ of $X_4$ and $X_5$ are close to zero, and even negative. According to the relevance coefficients, $X_4$ and $X_5$ are therefore irrelevant, even a bit harmful for the clustering process overall. As each additional feature increases the $SSW$, $X_4$ and $X_5$ should be excluded from the clustering process.